# Visualizing tools’ results

graph stuff:
- https://holoviews.org/gallery/demos/bokeh/network_graph.html
- https://holoviews.org/reference/elements/bokeh/Graph.html

In [ ]:
import holoviews as hv

from hv_anndata import ACCESSOR as A
from hv_anndata import register
from hv_anndata.interface import AdPath

register()

hv.extension("bokeh")

In [ ]:
from typing import Literal

import numpy as np
import scanpy as sc
from anndata import AnnData
from scanpy._utils import NeighborsView

In [ ]:
adata = sc.datasets.pbmc68k_reduced()
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)

{func}`scanpy.pl.embedding` (i.e. {func}`scanpy.pl.pca`, {func}`scanpy.pl.umap`, {func}`scanpy.pl.diffmap`, {func}`scanpy.pl.tsne`, {func}`scanpy.pl.draw_graph`)

missing:
- `add_outline`
- fast graph routines: https://github.com/holoviz/holoviews/issues/6667
- `legend_position` doesn’t work for graphs

missing convenience:
- `groups` to restrict to a subset easily

In [ ]:
# without edges, see `scatter` in Basic notebook


def draw_graph(
    adata: AnnData,
    node_vdims: AdPath | list[AdPath] | None = None,
    edge_vdim: Literal["distances", "connectivities"] = "connectivities",
    *,
    neighbors_key: str = "neighbors",
) -> hv.Graph:
    adata = adata.copy()
    adata.obs["cell index"] = range(adata.n_obs)
    # TODO: allow to use things like A.obsp["connectivities"]  # noqa: TD003
    ns = NeighborsView(adata, neighbors_key)
    coo = ns[edge_vdim].tocoo()
    nodes = hv.Nodes(
        adata,
        [A.obsm["X_umap"][0], A.obsm["X_umap"][1], A.obs["cell index"]],
        node_vdims,
    )
    return hv.Graph(((*coo.coords, coo.data), nodes), vdims=edge_vdim)


draw_graph(adata, [A.obs["bulk_labels"]]).opts(
    node_color=A.obs["bulk_labels"],
    node_cmap="tab10",
    edge_color="connectivities",
    aspect="square",
    legend_position="right",
)

{func}`scanpy.pl.ranking` ({func}`scanpy.pl.pca_loadings`, {func}`scanpy.pl.pca_variance_ratio`)

In [ ]:
def ranking(
    adata, dim: AdPath, n_points: int = 10, *, include_lowest: bool = True
) -> hv.Labels:
    def mk_plot(
        scores, labels, start: int, text_align: Literal["left", "right", "center"]
    ):
        rank = np.arange(start, start + len(labels))
        ds = dict(rank=rank, scores=scores, label=labels)
        lbls = hv.Labels(ds, ["rank", "scores"], ["label"]).opts(
            angle=90, text_align=text_align, xticks=0
        )
        return (
            lbls
            if text_align == "center"
            else lbls * hv.Points(ds, ["rank", "scores"], ["label"])
        )

    scores = dim(adata)
    [ax] = dim.axes
    labels = getattr(adata, ax).index
    idx = np.argsort(scores)

    idx_top, idx_bot = idx[-n_points:][::-1], idx[:n_points][::-1]
    top = mk_plot(scores[idx_top], labels[idx_top], 0, "right")
    if not include_lowest:
        return top

    mid = mk_plot([np.nanmean(scores)], ["⋯"], n_points, "center")
    bot = mk_plot(scores[idx_bot], labels[idx_bot], n_points + 1, "left")
    return top * bot * mid  # mid last so dots share color with text


ranking(adata, A.varm["PCs"][0]).opts(aspect=1.2)